# GAN: Generative Adversarial Networks

In [ ]:
import torch
import torch.nn as nn

from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.utils import save_image

import matplotlib.pyplot as plt

In [ ]:
latent_dim = 100

# 생성자(Generator) 클래스 정의
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        # 하나의 블록(block) 정의
        def block(input_dim, output_dim, normalize=True):
            layers = [nn.Linear(input_dim, output_dim)]
            if normalize:
                # 배치 정규화(batch normalization) 수행(차원 동일)
                layers.append(nn.BatchNorm1d(output_dim, 0.8))
            layers.append(nn.LeakyReLU(0.1, inplace=True))
            return layers

        # 생성자 모델은 연속적인 여러 개의 블록을 가짐
        self.model = nn.Sequential(
            *block(latent_dim, 128, normalize=True),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, 1 * 28 * 28),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), 1, 28, 28)
        return img

In [ ]:
# 판별자(Discriminator) 클래스 정의
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        # MLP 형태
        self.model = nn.Sequential(
            nn.Linear(1 * 28 * 28, 512),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    # 이미지에 대한 판별 결과를 반환
    def forward(self, img):
        flattened = img.view(img.size(0), -1)
        output = self.model(flattened)

        return output

In [ ]:
class MNIST_DataSet(torch.utils.data.Dataset):
    # 인자를 받아 인스턴스 변수를 초기화 
    # image_list - 이미지 전체 리스트
    def __init__(self, image_list, transform=None):
        super().__init__()
        self.image_list = image_list
        self.transform = transform

    # 데이터 셋의 길이를 리턴한다.
    def __len__(self):
        return len(self.image_list)
    
    # 학습 이미지를 리턴한다.
    def __getitem__(self, idx):
        image = self.image_list[idx]

        if self.transform:
          image = self.transform(image)

        return image

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),                                
                                transforms.Resize(28),
                                transforms.Normalize([0.5], [0.5])])

# GAN은 Test용 Dataset을 별도로 받을 필요가 없음
train_dataset = datasets.MNIST(root="./dataset", 
                               train=True, 
                               download=True,
                               transform=transform)

train = MNIST_DataSet(train_dataset.data.numpy(), transform=transform)

imageloader = torch.utils.data.DataLoader(train, 
                                          batch_size=1024, 
                                          shuffle=True, 
                                          num_workers=2)

In [ ]:
# 생성자(generator)와 판별자(discriminator) 초기화
generator = Generator()
discriminator = Discriminator()

# generator, discriminator를 cuda로 올린다
generator.cuda()
discriminator.cuda()

# 손실 함수(loss function) : Binary Cross Entropy Loss ( 진짜, 가짜 판별 : 이진 분류 )
# loss function을 cuda로 올린다
adversarial_loss = nn.BCELoss()
adversarial_loss.cuda()

# 학습률(learning rate) 설정
lr = 0.0002

# 생성자와 판별자를 위한 최적화 함수
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

In [ ]:
generator

Generator(
  (model): Sequential(
    (0): Linear(in_features=100, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1, inplace=True)
    (3): Linear(in_features=128, out_features=256, bias=True)
    (4): BatchNorm1d(256, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.1, inplace=True)
    (6): Linear(in_features=256, out_features=512, bias=True)
    (7): BatchNorm1d(512, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.1, inplace=True)
    (9): Linear(in_features=512, out_features=1024, bias=True)
    (10): BatchNorm1d(1024, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (11): LeakyReLU(negative_slope=0.1, inplace=True)
    (12): Linear(in_features=1024, out_features=784, bias=True)
    (13): Tanh()
  )
)

In [ ]:
discriminator

Discriminator(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): LeakyReLU(negative_slope=0.1, inplace=True)
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): LeakyReLU(negative_slope=0.1, inplace=True)
    (4): Linear(in_features=256, out_features=1, bias=True)
    (5): Sigmoid()
  )
)

In [ ]:
def plot_images(images):
    images = images.cpu()
    fig = plt.figure(figsize=(20, 20))

    for idx, image in enumerate(images):
        # 1, 28, 28 => 28, 28
        image = torch.squeeze(image)
        ax = fig.add_subplot(3, 3, idx+1)
        ax.imshow(image)

    fig.show()

In [ ]:
import time

n_epochs = 200 # 학습의 횟수(epoch) 설정
sample_interval = 100 # 몇 번의 배치(batch)마다 결과를 출력할 것인지 설정
start_time = time.time()
hist = {'d_loss' : [], 'g_loss' : []}

for epoch in range(n_epochs):
    for i, imgs in enumerate(imageloader):

        # 진짜(real) 이미지와 가짜(fake) 이미지에 대한 정답 레이블 생성 (cuda로)
        real = torch.cuda.FloatTensor(imgs.size(0), 1).fill_(1.0) # 진짜(real): 1
        fake = torch.cuda.FloatTensor(imgs.size(0), 1).fill_(0.0) # 가짜(fake): 0

        real_imgs = imgs.cuda()

        """ 생성자(generator)를 학습합니다. """
        optimizer_G.zero_grad()

        # 랜덤 노이즈(noise) 샘플링 (cuda로)
        z = torch.normal(mean=0, std=1, size=(imgs.shape[0], latent_dim)).cuda()

        # 이미지 생성
        generated_imgs = generator(z)

        # 생성자(generator)의 손실(loss) 값 계산
        g_loss = adversarial_loss(discriminator(generated_imgs), real)

        # 생성자(generator) 업데이트
        g_loss.backward()
        optimizer_G.step()

        """ 판별자(discriminator)를 학습합니다. """
        optimizer_D.zero_grad()

        # 판별자(discriminator)의 손실(loss) 값 계산
        real_loss = adversarial_loss(discriminator(real_imgs), real)
        fake_loss = adversarial_loss(discriminator(generated_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        # 판별자(discriminator) 업데이트
        d_loss.backward()
        optimizer_D.step()

        done = epoch * len(imageloader) + i
        if done % sample_interval == 0:
            plot_images(generated_imgs.data[:9])
            # 생성된 이미지 중에서 25개만 선택하여 5 X 5 격자 이미지에 출력
            # save_image(generated_imgs.data[:25], f"{done}.png", nrow=5, normalize=True)

    # 하나의 epoch이 끝날 때마다 로그(log) 출력, history에 loss값 저장
    print(f"[Epoch {epoch + 1}/{n_epochs}] [D loss: {d_loss.item():.6f}] [G loss: {g_loss.item():.6f}] [Elapsed time: {time.time() - start_time:.2f}s]")
    
    hist['d_loss'].append(d_loss.item())
    hist['g_loss'].append(g_loss.item())

[Epoch 1/200] [D loss: 0.542586] [G loss: 0.629575] [Elapsed time: 3.14s]
[Epoch 2/200] [D loss: 0.524990] [G loss: 0.733642] [Elapsed time: 6.25s]
[Epoch 3/200] [D loss: 0.563144] [G loss: 0.756645] [Elapsed time: 9.32s]
[Epoch 4/200] [D loss: 0.495023] [G loss: 1.636221] [Elapsed time: 12.38s]
[Epoch 5/200] [D loss: 0.427246] [G loss: 0.765303] [Elapsed time: 15.35s]
[Epoch 6/200] [D loss: 0.471336] [G loss: 0.979601] [Elapsed time: 18.41s]
[Epoch 7/200] [D loss: 0.504689] [G loss: 1.486079] [Elapsed time: 21.45s]
[Epoch 8/200] [D loss: 0.411939] [G loss: 0.995320] [Elapsed time: 24.46s]
[Epoch 9/200] [D loss: 0.465572] [G loss: 1.172309] [Elapsed time: 27.51s]
[Epoch 10/200] [D loss: 0.542683] [G loss: 1.625595] [Elapsed time: 30.50s]
[Epoch 11/200] [D loss: 0.380779] [G loss: 0.961354] [Elapsed time: 33.52s]
[Epoch 12/200] [D loss: 0.482408] [G loss: 1.112660] [Elapsed time: 36.61s]
[Epoch 13/200] [D loss: 0.479489] [G loss: 1.615893] [Elapsed time: 39.69s]
[Epoch 14/200] [D loss: 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until


[Epoch 34/200] [D loss: 0.331809] [G loss: 1.866219] [Elapsed time: 104.12s]
[Epoch 35/200] [D loss: 0.287388] [G loss: 1.548493] [Elapsed time: 107.14s]
[Epoch 36/200] [D loss: 0.244702] [G loss: 1.278029] [Elapsed time: 110.21s]
[Epoch 37/200] [D loss: 0.310319] [G loss: 1.287450] [Elapsed time: 113.20s]
[Epoch 38/200] [D loss: 0.414342] [G loss: 3.272338] [Elapsed time: 116.28s]
[Epoch 39/200] [D loss: 0.291906] [G loss: 1.267127] [Elapsed time: 119.39s]
[Epoch 40/200] [D loss: 0.316152] [G loss: 1.353804] [Elapsed time: 122.40s]
[Epoch 41/200] [D loss: 0.270755] [G loss: 1.345102] [Elapsed time: 125.52s]
[Epoch 42/200] [D loss: 0.336491] [G loss: 1.006948] [Elapsed time: 128.45s]
[Epoch 43/200] [D loss: 0.277623] [G loss: 3.629857] [Elapsed time: 131.51s]
[Epoch 44/200] [D loss: 0.272414] [G loss: 2.946200] [Elapsed time: 134.51s]
[Epoch 45/200] [D loss: 0.197941] [G loss: 1.604400] [Elapsed time: 137.69s]
[Epoch 46/200] [D loss: 0.373669] [G loss: 0.821809] [Elapsed time: 140.80s]

학습속도 : Epoch당 약 15 ~ 20초 (GPU기준)

In [ ]:
def history(hist):
  fig = plt.figure(figsize=(20, 10))
  ax = fig.add_subplot(1, 2, 1)
  ax.plot(range(1, n_epochs+1), hist['d_loss'], color="red", label="d loss")
  ax.set_title("D Loss")
  ax.legend()

  ax = fig.add_subplot(1, 2, 2)
  ax.plot(range(1, n_epochs+1), hist['g_loss'], color="blue", label="g loss")
  ax.set_title("G Loss")
  ax.legend()

  fig.show()

In [ ]:
history(hist)

In [ ]:
# 랜덤 노이즈(noise) 샘플링 (cuda로)
z = torch.normal(mean=0, std=1, size=(imgs.shape[0], latent_dim)).cuda()

# 이미지 생성
generated_imgs = generator(z)

In [ ]:
plot_images(generated_imgs.data[:9])